## model.MoveActivity

This notebook shows the basic contours of a first OpenCLSim simulation. We use OpenCLSim's __model.MoveActivity__. We define a __from_site__ location and a __vessel01__ that moves from there to a __to_site__. No volume is shifted yet.

#### 0. Import libraries

In [1]:
import datetime, time
import simpy

import shapely.geometry
import pandas as pd

import openclsim.core as core
import openclsim.model as model
import openclsim.plot as plot

#### 1. Initialise simpy environment

In [2]:
# setup environment
simulation_start = 0
my_env = simpy.Environment(initial_time=simulation_start)

#### 2. Define object classes

In [3]:
# create a Site object based on desired mixin classes
Site = type(
    "Site",
    (
        core.Identifiable,
        core.Log,
        core.Locatable,
        core.HasContainer,
        core.HasResource,
    ),
    {},
)

# create a TransportProcessingResource object based on desired mixin classes
TransportProcessingResource = type(
    "TransportProcessingResource",
    (
        core.Processor,
        core.ContainerDependentMovable,
        core.Identifiable,
        core.HasResource,
    ),
    {},
)

#### 3. Create objects
##### 3.1. Create site object(s)

In [4]:
# prepare input data for from_site (note: in this example we only need the start location)
location_from_site = shapely.geometry.Point(4.18055556, 52.18664444)

# prepare input data for to_site
location_to_site = shapely.geometry.Point(4.25222222, 52.11428333)
data_to_site = {"env": my_env,
                "name": "to_site",
                "geometry": location_to_site,
                "capacity": 100,
                "level": 100
               }
# instantiate to_site 
to_site = Site(**data_to_site)

##### 3.2. Create vessel object(s)

In [5]:
# prepare input data for vessel_01
data_vessel01 = {"env": my_env,
                 "name": "vessel01",
                 "geometry": location_from_site, 
                 "capacity": 5,
                 "compute_v": lambda x: 10
               }
# instantiate vessel_01 
vessel01 = TransportProcessingResource(**data_vessel01)


##### 3.3 Create activity/activities

In [6]:
# initialise registry
registry = {}

In [7]:
activity = model.MoveActivity(
    env=my_env,
    name="Move activity",
    registry=registry,
    mover=vessel01,
    destination=to_site,
)

#### 4. Register processes and run simpy

In [8]:
# initate the simpy processes defined in the 'move activity' and run simpy
model.register_processes([activity])
my_env.run()

#### 5. Inspect results
##### 5.1 Inspect logs
We can now inspect the logs. Since the model only contains a move activity not volume was shifted.

In [9]:
display(plot.get_log_dataframe(activity))

,Activity,Timestamp,ActivityState
0,116be63e-1901-496a-9487-dcf5756e2e8f,1970-01-01 00:00:00.000000,START
1,116be63e-1901-496a-9487-dcf5756e2e8f,1970-01-01 00:15:42.824591,STOP


Note that the log shows only uuids, and not human readable names. Add a list of activities for which you want the unique uuids to be mapped to the (not necesassily unique) human readable names. So you have to suppply the activity object twice.

In [10]:
display(plot.get_log_dataframe(activity, [activity]))

,Activity,Timestamp,ActivityState
0,Move activity,1970-01-01 00:00:00.000000,START
1,Move activity,1970-01-01 00:15:42.824591,STOP


In [11]:
display(plot.get_log_dataframe(vessel01, [activity]))

,Activity,Timestamp,ActivityState,container level,geometry
0,Move activity,1970-01-01 00:00:00.000000,START,0.0,POINT (4.18055556 52.18664444)
1,Move activity,1970-01-01 00:15:42.824591,STOP,0.0,POINT (4.25222222 52.11428333)


Observe that there was movement. There was no amount shifting.